## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
# model=keras.applications.ResNet50(include_top=False, weights='imagenet',
#                                   input_tensor=input_tensor,
#                                   pooling=None, classes=10)
'''Resnet 50 架構'''
# model=keras.applications.ResNet50(include_top=False, weights='imagenet',
#                                   input_tensor=input_tensor,
#                                   pooling=None, classes=10)
# model.summary()


Using TensorFlow backend.


'Resnet 50 架構'

## 添加層數

In [2]:
x = model.output

'''可以參考Cifar10實作章節'''
x = GlobalAveragePooling2D()(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))



Model深度： 177


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


## 鎖定特定幾層不要更新權重

In [0]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [5]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 76s 2ms/step - loss: 1.6118 - acc: 0.5656
Epoch 2/100
50000/50000 [==============================] - 65s 1ms/step - loss: 1.2707 - acc: 0.6467
Epoch 3/100
50000/50000 [==============================] - 65s 1ms/step - loss: 1.2012 - acc: 0.6874
Epoch 4/100
50000/50000 [==============================] - 64s 1ms/step - loss: 1.0870 - acc: 0.7181
Epoch 5/100
50000/50000 [==============================] - 63s 1ms/step - loss: 1.0085 - acc: 0.7440
Epoch 6/100
50000/50000 [==============================] - 63s 1ms/step - loss: 0.6933 - acc: 0.7692
Epoch 7/100
50000/50000 [==============================] - 64s 1ms/step - loss: 0.5600 - acc: 0.8144
Epoch 8/100
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4983 - acc: 0.8388
Epoch 9/100
50000/50000 [==============================] - 63s 1ms/step - loss: 0.4463 - acc: 

In [4]:
'''Xception 架構'''
model=keras.applications.Xception(include_top=False, weights='imagenet',
                                  input_tensor=input_tensor,
                                  pooling=None, classes=10)
model.summary()











83689472/83683744 [==============================] - 6s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
________________

In [5]:
x = model.output

'''可以參考Cifar10實作章節'''
x = Flatten()(x)
predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 134


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  """


In [0]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [7]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 50s 1ms/step - loss: 1.3768 - acc: 0.5165
Epoch 2/100
50000/50000 [==============================] - 41s 827us/step - loss: 0.9830 - acc: 0.6575
Epoch 3/100
50000/50000 [==============================] - 41s 823us/step - loss: 0.8656 - acc: 0.6995
Epoch 4/100
50000/50000 [==============================] - 41s 822us/step - loss: 0.7816 - acc: 0.7279
Epoch 5/100
50000/50000 [==============================] - 41s 821us/step - loss: 0.7035 - acc: 0.7545
Epoch 6/100
50000/50000 [==============================] - 41s 821us/step - loss: 0.6278 - acc: 0.7807
Epoch 7/100
50000/50000 [==============================] - 41s 822us/step - loss: 0.5607 - acc: 0.8051
Epoch 8/100
50000/50000 [==============================] - 41s 823us/step - loss: 0.4936 - acc: 0.8266
Epoch 9/100
50000/50000 [==============================] - 41s 829us/step - loss